# Demo for Classification Model Task

The current notebook is going to be a demo of the classification model capabilities, while also being used as the initial steps for the creation of the pipeline of the deployed version.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pickle, joblib


sys.path.insert(0, '..')

import support_functions

## Simple Mock Gait Cycle data from Gyroscope

In [2]:

def simulate_custom_gait_signal(duration_sec: int=60 , fs: int=100, freq: int=1.2, stroke: bool=False, noise_std: float=0.05):
    '''
    Simulate custom gait signals for stroke and healthy individuals.
    Args:
        duration_sec (int): Duration of the signal in seconds.
        fs (int): Sampling frequency in Hz.
        freq (float): Frequency of the sine wave.
        stroke (bool): If True, simulate stroke condition.
        noise_std (float): Standard deviation of the noise.
    '''
    t = np.linspace(0, duration_sec, int(duration_sec * fs))
    
    # Define target peak values
    if stroke:
        max_val   = 300
        amp_right = 200
    else:
        max_val   = 450
        amp_right = 420
    
    # Simulate left and right signals (right affected in stroke)
    left_z  = (max_val / 2) * np.sin(2 * np.pi * freq * t) + np.random.normal(0, noise_std * max_val, size=t.shape)
    right_z = (amp_right / 2) * np.sin(2 * np.pi * freq * t + (0.2 if stroke else 0)) + np.random.normal(0, noise_std * amp_right, size=t.shape)

    # Return DataFrame
    dataframe = pd.DataFrame({
        'timestamp (+0700)': pd.date_range('2024-01-01 00:00:00', periods=len(t), freq=f'{int(1000/fs)}ms'),
        'left-z-axis (deg/s)': left_z,
        'right-z-axis (deg/s)': right_z
    })
    return dataframe

# Generate example stroke and healthy signals
stroke_df = simulate_custom_gait_signal(stroke=True)
healthy_df = simulate_custom_gait_signal(stroke=False)

## Load the Linear Regression Classification Model 

In [ ]:
model = joblib.load('/Users/stamatiosorphanos/Documents/GaitRehab/models/classification_logistic_regression.pkl')
print('Input features:', model.n_features_in_)
print('Classes:', model.classes_)

### Healthy Subject Prediction 

In [ ]:
model_input = support_functions.time_domain_features(healthy_df)

[-270.06457039727627,
 -302.44746407166417,
 265.3970246795004,
 295.76964360518025]

,timestamp (+0700),left-z-axis (deg/s),right-z-axis (deg/s)
0,2024-01-01 00:00:00.000,20.363230,15.881663
1,2024-01-01 00:00:00.010,34.010398,-21.816337
2,2024-01-01 00:00:00.020,-6.300542,10.403566
3,2024-01-01 00:00:00.030,81.588652,29.681843
4,2024-01-01 00:00:00.040,36.898967,44.694441
...,...,...,...
5995,2024-01-01 00:00:59.950,-42.455550,-47.061242
5996,2024-01-01 00:00:59.960,-24.541417,-58.268253
5997,2024-01-01 00:00:59.970,-33.688494,-43.956626
5998,2024-01-01 00:00:59.980,-51.963879,9.170632


Input features: 4
Classes: [0 1]
